In [163]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer

from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import xgboost as xgb
import string, re
import nltk
import tensorflow as tf
import time

from nltk.corpus import stopwords   
from nltk.stem.porter import PorterStemmer
from gensim.models import KeyedVectors
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
import string,re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from gensim.models import KeyedVectors
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [2]:
sw=stopwords.words('english')
ps=PorterStemmer()
def cleatext(text):
    
    text=text.translate(text.maketrans("","",string.punctuation))
    text=[ps.stem(w) for w in text.split(' ') if w not in sw]
    text=' '.join(text)
    return text

In [19]:


def readdata(filename,fg):
    df=pd.read_csv(filename)
    review_title=df['Review Title']
    review_text=df['Review Text']
    topic=[]

    review=df['Review Text']+df['Review Title']
    if fg==1:
        topic=df['topic']
    review_text=review_text.apply(cleatext)
    review_title=review_title.apply(cleatext)
    review=review.apply(cleatext)
    
    return df,review,review_text,review_title,topic

In [20]:
df,review,review_text,review_title,topic=readdata('train.csv',1)

In [134]:

cv=CountVectorizer(max_features=1000)
cv.fit(review_title)


CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=1000, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [135]:
x1=cv.transform(review_title)

In [136]:
tfdf=TfidfVectorizer(max_features=2000)

In [137]:
tfdf.fit(review_text)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [138]:
x2=tfdf.transform(review_text)

In [139]:
X=np.hstack([x1.toarray(),x2.toarray()])

In [140]:
X.shape

(5959, 3000)

In [141]:
def addextra(review,X):
    
    snl=SentimentIntensityAnalyzer()
    star=review.apply(lambda x:1 if "*" in x else 0)

    smiley=review.apply(lambda x:1 if ":)" in x else 0)
    sad=review.apply(lambda x:1 if ":(" in x else 0)
    exp=review.apply(lambda x:1 if "!" in x else 0)
    hash1=review.apply(lambda x:1 if "#" in x else 0)
    comma1=review.apply(lambda x:1 if "," in x else 0)
    dollar=review.apply(lambda x:1 if "$" in x else 0)

    length1=review.apply(lambda x:len(x))
    compound_polarity=review.apply(lambda x:snl.polarity_scores(x)['compound'])
    
    star=np.array(star).reshape(-1,1)
    smiley=np.array(smiley).reshape(-1,1)
    sad=np.array(sad).reshape(-1,1)
    exp=np.array(exp).reshape(-1,1)
    hash1=np.array(hash1).reshape(-1,1)
    comma1=np.array(comma1).reshape(-1,1)
    dollar=np.array(dollar).reshape(-1,1)
    length1=np.array(length1).reshape(-1,1)
    compound_polarity=np.array(compound_polarity).reshape(-1,1)
    print(length1.shape)
    print(X.shape)
    X=np.hstack([X,star,smiley,sad,exp,hash1,comma1,dollar,length1,compound_polarity])
    print(X.shape)
    return X

def addextra1(review,X):
    
    snl=SentimentIntensityAnalyzer()
  

    length1=review.apply(lambda x:len(x))
    compound_polarity=review.apply(lambda x:snl.polarity_scores(x)['compound'])
    
    length1=np.array(length1).reshape(-1,1)
    compound_polarity=np.array(compound_polarity).reshape(-1,1)

    X=np.hstack([X,length1,compound_polarity])
    print(X.shape)
    return X
    
    

In [142]:
X=addextra1(review,X)
X.shape

(5959, 3002)


(5959, 3002)

In [143]:
y=topic

In [144]:
sc=StandardScaler()
X=sc.fit_transform(X)

In [145]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [146]:


xg=xgb.XGBClassifier()

In [147]:
xg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=True, subsample=1)

In [148]:
y_pred=xg.predict(X_test)

In [149]:
accuracy_score(y_pred,y_test)

0.5419463087248322

In [164]:
xg1=xgb.XGBClassifier()

In [167]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }

folds = 3
param_comb = 5

In [168]:
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

In [ ]:
    
#should evaluate by train_eval instead of the full dataset
clf = GridSearchCV(xg1, params, n_jobs=4, 
                   cv=5, 
                   
                   verbose=2, refit=True)



clf.fit(X_train, y_train) 
 
#trust your CV! 
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])

Fitting 5 folds for each of 405 candidates, totalling 2025 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 78.6min
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed: 333.6min
[Parallel(n_jobs=4)]: Done 357 tasks      | elapsed: 691.1min


In [ ]:
def buildmodel():
    xg=xgb.XGBClassifier()
    xg.fit(X_train,y_train)
    y_pred=xg.predict(X_test)
    print(accuracy_score(y_pred,y_test))
    return xg

In [ ]:
xg=buildmodel()

In [153]:
dt=pd.read_csv('test.csv')
dt_review_text=dt['Review Text']
dt_review_title=dt['Review Title']

In [154]:
dt,review_val,review_text_val,review_title_val,topic_val=readdata('test.csv',0)

xt1=cv.transform(review_title_val)
xt2=tfdf.transform(review_text_val)
Xt=np.hstack([xt1.toarray(),xt2.toarray()])
Xt=addextra1(review_val,Xt)

(2553, 3002)


In [155]:
Xt=sc.fit_transform(Xt)

In [156]:
y_pred=xg.predict(Xt)

In [157]:

tp=pd.DataFrame({'Review Text':dt_review_text,'Review Title':dt_review_title,'topic':y_pred})
tp.to_csv('Submission_XGBoost9.csv',index=False)